In [36]:
import numpy as np
import scipy
from pandas import *
from ggplot import *
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sys
import scipy.stats as sstat

df = pandas.read_csv('turnstile_weather_v2.csv')
df.head()

##make sure to run this before any other sections when runnning notebook

UNIT     DATEn     TIMEn  ENTRIESn   EXITSn  ENTRIESn_hourly  \
0  R003  05-01-11  00:00:00   4388333  2911002                0   
1  R003  05-01-11  04:00:00   4388333  2911002                0   
2  R003  05-01-11  12:00:00   4388333  2911002                0   
3  R003  05-01-11  16:00:00   4388333  2911002                0   
4  R003  05-01-11  20:00:00   4388333  2911002                0   

   EXITSn_hourly             datetime  hour  day_week     ...       pressurei  \
0              0  2011-05-01 00:00:00     0         6     ...           30.22   
1              0  2011-05-01 04:00:00     4         6     ...           30.25   
2              0  2011-05-01 12:00:00    12         6     ...           30.28   
3              0  2011-05-01 16:00:00    16         6     ...           30.26   
4              0  2011-05-01 20:00:00    20         6     ...           30.28   

  rain  tempi  wspdi meanprecipi  meanpressurei  meantempi  meanwspdi  \
0    0   55.9    3.5           0         30.258      55.98       7.86   
1    0   52.0    3.5           0         30.258      55.98       7.86   
2    0   62.1    6.9           0         30.258      55.98       7.86   
3    0   57.9   15.0           0         30.258      55.98       7.86   
4    0   52.0   10.4           0         30.258      55.98       7.86   

   weather_lat  weather_lon  
0    40.700348   -73.887177  
1    40.700348   -73.887177  
2    40.700348   -73.887177  
3    40.700348   -73.887177  
4    40.700348   -73.887177  

[5 rows x 27 columns]

In [2]:
#df.describe() shows 42649 rows and since rain mean is 0.224741 that means it does not rain on the majority of days.
df.groupby('rain').describe().transpose().loc[:,(slice(None),['count','mean']),]

rain                 0                      1                 
                 count             mean count             mean
ENTRIESn         33064  28158345.474050  9585  28009333.335107
ENTRIESn_hourly  33064      1845.539439  9585      2028.196035
EXITSn           33064  19906411.104767  9585  19744097.584351
EXITSn_hourly    33064      1333.111451  9585      1459.373918
day_week         33064         2.998064  9585         2.587167
fog              33064         0.002147  9585         0.036307
hour             33064        10.049359  9585        10.037767
latitude         33064        40.724497  9585        40.725167
longitude        33064       -73.940368  9585       -73.940349
meanprecipi      33064         0.000008  9585         0.020520
meanpressurei    33064        29.997082  9585        29.881459
meantempi        33064        64.304694  9585        58.961158
meanwspdi        33064         6.561112  9585         8.193031
precipi          33064         0.000008  9585         0.020520
pressurei        33064        29.997082  9585        29.881459
tempi            33064        64.304694  9585        58.961158
weather_lat      33064        40.728433  9585        40.728978
weather_lon      33064       -73.938697  9585       -73.938680
weekday          33064         0.682616  9585         0.824204
wspdi            33064         6.561112  9585         8.193031

In [3]:
#Mann Whitney u Test
norain = df[df['rain']==0]['ENTRIESn_hourly']
rain = df[df['rain']==1]['ENTRIESn_hourly']

U,p=scipy.stats.mannwhitneyu(rain,norain)

print "MannWhitneyU test U=", U,", p=",p

print "\nNeeded to calculate p manually because it is not calculating correctly"
m_u = len(rain)*len(norain)/2

sigma_u = np.sqrt(len(rain)*len(norain)*(len(rain)+len(norain)+1)/12)
z = (U - m_u)/sigma_u
p = scipy.stats.norm.cdf(z,scale=1)


print "p value=",p

print "P times two because two tailed=", p*2

print "\nNo Rain Entries Mean:", np.mean(norain)

print "Rain Entries Mean:", np.mean(rain)



MannWhitneyU test U= 153635120.5 , p= nan

Needed to calculate p manually because it is not calculating correctly
p value= 2.74134693571e-06
P times two because two tailed= 5.48269387142e-06

No Rain Entries Mean: 1845.53943866
Rain Entries Mean: 2028.19603547


In [26]:
"""
OLS experiment This function takes in our pandas turnstile weather dataframe, and returns a set of predicted ridership values,
based on the other information in the dataframe.  

In exercise 3.5 we used Gradient Descent in order to compute the coefficients
theta used for the ridership prediction. Here you should attempt to implement 
another way of computing the coeffcients theta. You may also try using a reference implementation such as: 

One of the advantages of the statsmodels implementation is that it gives you
easy access to the values of the coefficients theta. This can help you infer relationships 
between variables in the dataset. 
"""
features = df[['rain', 'hour', 'tempi', 'pressurei']]

dummy=pandas.get_dummies(df['UNIT'],prefix='unit')
features=features.join(dummy)

values = df['ENTRIESn_hourly']

prediction = sm.OLS(values, features).fit()
    
print prediction.summary()
print prediction.params
print prediction.predict()


                            OLS Regression Results                            
Dep. Variable:        ENTRIESn_hourly   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.457
Method:                 Least Squares   F-statistic:                     148.5
Date:                Thu, 16 Apr 2015   Prob (F-statistic):               0.00
Time:                        22:25:16   Log-Likelihood:            -3.8817e+05
No. Observations:               42649   AIC:                         7.768e+05
Df Residuals:                   42405   BIC:                         7.789e+05
Df Model:                         243                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
rain         183.8770     28.383      6.478      0.0

# Visualization

In [32]:
overall=norain.plot(kind='hist',bins=200,label='No Rain',xlim=(0,10000),ylim=(0,6100))
overall=rain.plot(kind='hist',bins=200,label='Rain',xlim=(0,10000),ylim=(0,6100))

overall.legend()
overall.set_xlabel('Entries Hourly')
overall.set_ylabel('Fequency')
overall.set_title('Overall Entries Hourly vs Frequency')

<img src="overall.png">

#Further Analysis Experimentation

In [435]:
#SAMPLE TESTING ANALYSIS
#Diagram above made me think what if we remove outliers where no rain had significant advantange 
##because it doesn't rain most of the time, then we take a random sample of that data...

samplesize=2000
outliers=2500
run=5
x=0

while x<run:
    NOrain = df[(df['rain']==0) & (df['ENTRIESn_hourly']>outliers)]['ENTRIESn_hourly']
    YESrain = df[(df['rain']==1) & (df['ENTRIESn_hourly']>outliers)]['ENTRIESn_hourly']
    print "\nMannWhitneyU test U=",U,",p=",p
    print "Entire Population: Observations, (min,max),       mean,              variance,             skewness,       kurtosis)"
    print "No Rain        ", sstat.describe(NOrain)
    print "Rain           ", sstat.describe(YESrain)

    U,p=scipy.stats.mannwhitneyu(YESrain,NOrain)

    
    print "Sample:"
    sampleNOrain=np.random.choice(NOrain, size=samplesize, replace=False)
    sampleYESrain=np.random.choice(YESrain, size=samplesize, replace=False)


    print "No Rain=       ", sstat.describe(sampleNOrain)
    print "Rain=          ", sstat.describe(sampleYESrain)
    x=x+1


MannWhitneyU test U= 8221620.0 ,p= 0.0271488390182
Entire Population: Observations, (min,max),       mean,              variance,             skewness,       kurtosis)
No Rain         (7249, (2501.0, 32814.0), 5692.6536073941234, 17226402.214018784, 2.6838871183064303, 8.985905002672501)
Rain            (2330, (2502.0, 32289.0), 5936.6197424892707, 20245854.374019686, 2.6559847657070312, 8.590829087194098)
Sample:
No Rain=        (2000, (2503.0, 31125.0), 5655.2719999999999, 16795812.304168083, 2.6222476630517453, 8.476186829533487)
Rain=           (2000, (2502.0, 32289.0), 5931.1925000000001, 20464736.221554525, 2.6887805499625217, 8.783258805199372)

MannWhitneyU test U= 8221620.0 ,p= 0.0271488390182
Entire Population: Observations, (min,max),       mean,              variance,             skewness,       kurtosis)
No Rain         (7249, (2501.0, 32814.0), 5692.6536073941234, 17226402.214018784, 2.6838871183064303, 8.985905002672501)
Rain            (2330, (2502.0, 32289.0), 5936.61

In [476]:
samplesize=2000
outliers=2500
run=50
x=0

while x<run:
    NOrain = df[(df['rain']==0) & (df['ENTRIESn_hourly']>outliers)]['ENTRIESn_hourly']
    YESrain = df[(df['rain']==1) & (df['ENTRIESn_hourly']>outliers)]['ENTRIESn_hourly']
    U,p=scipy.stats.mannwhitneyu(YESrain,NOrain)
    print "Population Mean", "No Rain=",np.mean(NOrain), "Rain=",np.mean(YESrain),"MannWhitneyU test U=",U,",p=",p,
    sampleNOrain=np.random.choice(NOrain, size=samplesize, replace=False)
    sampleYESrain=np.random.choice(YESrain, size=samplesize, replace=False)
    print "\n  Sample Mean   No Rain=", np.mean(sampleNOrain),"|",np.min(sampleNOrain),"/",np.max(sampleNOrain),"     Rain=", np.mean(sampleYESrain),np.min(sampleYESrain),"/",np.max(sampleYESrain)
    if np.mean(sampleNOrain)<np.mean(sampleYESrain):
        print "Rain Day-Higher average ridership in sample mean!\n"
    else:
        print "NO Rain Day-Higher average ridership in sample mean!\n"
    
    x=x+1

Population Mean No Rain= 5692.65360739 Rain= 5936.61974249 MannWhitneyU test U= 8221620.0 ,p= 0.0271488390182 
  Sample Mean   No Rain= 5805.22 | 2501.0 / 32814.0      Rain= 5898.382 2502.0 / 32289.0
Rain Day-Higher average ridership in sample mean!

Population Mean No Rain= 5692.65360739 Rain= 5936.61974249 MannWhitneyU test U= 8221620.0 ,p= 0.0271488390182 
  Sample Mean   No Rain= 5628.83 | 2503.0 / 31276.0      Rain= 5990.5375 2502.0 / 32289.0
Rain Day-Higher average ridership in sample mean!

Population Mean No Rain= 5692.65360739 Rain= 5936.61974249 MannWhitneyU test U= 8221620.0 ,p= 0.0271488390182 
  Sample Mean   No Rain= 5589.3095 | 2501.0 / 31306.0      Rain= 5965.992 2502.0 / 32289.0
Rain Day-Higher average ridership in sample mean!

Population Mean No Rain= 5692.65360739 Rain= 5936.61974249 MannWhitneyU test U= 8221620.0 ,p= 0.0271488390182 
  Sample Mean   No Rain= 5632.258 | 2502.0 / 32814.0      Rain= 5931.218 2502.0 / 32289.0
Rain Day-Higher average ridership in sample

In [487]:
run=2000
x=0
countRainMeanHigher=0
countNoRainMeanHigher=0

while x<run:
    if np.mean(sampleNOrain)<np.mean(sampleYESrain):
        countRainMeanHigher=countRainMeanHigher+1
    else:
        countNoRainMeanHigher=countNoRainMeanHigher+1
    x=x+1

print "Rain Mean Higher:",countRainMeanHigher,"\nNo Rain Mean Higher:",countNoRainMeanHigher

Rain Mean Higher: 2000 
No Rain Mean Higher: 0


In [444]:
Running this code numerous times shows rain mean is higher majority of the time on the random sample.

SyntaxError: invalid syntax (<ipython-input-444-d2ab8f7a7e4d>, line 1)

In [35]:
main=ggplot(df, aes(x='ENTRIESn_hourly',fill='rain'))+geom_histogram(binwidth=200)+facet_wrap('hour')\
+xlim(0,4000)+ylim(0,5000)+xlab('Entries Hourly')+ylab('Frequency')+ggtitle('Entries Frequency By Hours- Each Plot Refers to Hours Group (Red=No Rain, Blue=Rain)')\
+theme(axis_text_x  = element_text(angle = 20, hjust = 1))

overall=norain.plot(kind='hist',bins=200,label='No Rain',xlim=(0,10000),ylim=(0,6100))
overall=rain.plot(kind='hist',bins=200,label='Rain',xlim=(0,10000),ylim=(0,6100))

overall.legend()
overall.set_xlabel('Entries Hourly')
overall.set_ylabel('Fequency')
overall.set_title('Overall Entries Hourly vs Frequency')

print main

<ggplot: (36651517)>


<img src="freqperhr.png">

In [28]:
one=ggplot(df, aes('hour','ENTRIESn_hourly', color='rain'))+geom_point()+geom_line()\
+ggtitle('Hourly Entries Per Hour of Day(Red=Rain,Light Blue=No Rain)')\
+xlab('Hour of Day')+ylab('Entries')+xlim(0,23)+ylim(0,33000)+scale_x_continuous(breaks=range(0,24))
print one


<ggplot: (29526132)>


First plot attempt at a line plot of Hourly entries, the lines should be calculating overall ridership which shows here is highest when it doesnt rain around the 20th hour. We also see the trend of ridership going down when it rains from 12 to 16. Lets adjust the chart type and see some other way to view the data.

In [9]:
two=ggplot(df, aes('hour','', fill='rain'))\
+geom_bar(aes(x='hour',weight='ENTRIESn_hourly'))\
+ggtitle('Cumulative Entries Per Hour of Day(Red=No Rain, Blue=Rain)')\
+xlab('Hour of Day')+ylab('Entries By Hour')+xlim(-1,21)+ylim(0,25000000)+scale_x_continuous(breaks=range(0,21))

print two

<ggplot: (-897025466)>


This shows total ridership on rainy and non rainy days with color, but there are more non rainy days then rainy.

<img src="fig2.png">

In [10]:
three=ggplot(df, aes('hour','EXITSn_hourly', fill='rain'))\
+geom_bar(aes(x='hour',weight='EXITSn_hourly'))\
+ggtitle('Cumulative Exits Per Hour of Day(Red=No Rain, Blue=Rain)')\
+xlab('Hour of Day')+ylab('Exits By Hour')+xlim(-1,21)+ylim(0,25000000)+scale_x_continuous(breaks=range(0,21))

print three

<ggplot: (-897314772)>


Here we can see the exits are pretty close to the entries in the graph above.

<img src="fig3.png">

In [92]:
'''experiment with linear regression exercise'''

def normalize_features(array):
   """
   Normalize the features in the data set.
   """
   array_normalized = (array-array.mean())/array.std()
   mu = array.mean()
   sigma = array.std()

   return array_normalized, mu, sigma

def compute_cost(features, values, theta):
    """
    Compute the cost function given a set of features / values, 
    and the values for our thetas.
    
    This can be the same code as the compute_cost function in the lesson #3 exercises,
    but feel free to implement your own.
    """
    
    m = len(values)
    sum_of_square_errors = np.square(np.dot(features, theta) - values).sum()
    cost = sum_of_square_errors / (2*m)

    return cost

def gradient_descent(features, values, theta, alpha, num_iterations):
    """
    Perform gradient descent given a data set with an arbitrary number of features.
    
    This can be the same gradient descent code as in the lesson #3 exercises,
    but feel free to implement your own.
    """
    
    m = len(values)
    cost_history = []

    for i in range(num_iterations):
        predictedValues = np.dot(features,theta)
        theta = theta - alpha/m * (np.dot((predictedValues - values), features))
        cost = compute_cost(features, values, theta)
        cost_history.append(cost)
    return theta, pandas.Series(cost_history)

def predictions(df):
    '''
    The NYC turnstile data is stored in a pandas dataframe called weather_turnstile.
    Using the information stored in the dataframe, let's predict the ridership of
    the NYC subway using linear regression with gradient descent.
    
    Your prediction should have a R^2 value of 0.20 or better.
    You need to experiment using various input features contained in the dataframe. 
    We recommend that you don't use the EXITSn_hourly feature as an input to the 
    linear model because we cannot use it as a predictor: we cannot use exits 
    counts as a way to predict entry counts. 
    
    If you'd like to view a plot of your cost history, uncomment the call to 
    plot_cost_history below. The slowdown from plotting is significant, so if you 
    are timing out, the first thing to do is to comment out the plot command again.
    
    If you receive a "server has encountered an error" message, that means you are 
    hitting the 30-second limit that's placed on running your program. Try using a 
    smaller number for num_iterations if that's the case.
    
    If you are using your own algorithm/models, see if you can optimize your code so 
    that it runs faster.
    '''
    # Select Features (try different features!)
    features = df[['rain', 'precipi', 'hour', 'meanwspdi']]
    
    # Add UNIT to features using dummy variables
    dummy_units = pandas.get_dummies(df['UNIT'], prefix='unit')
    features = features.join(dummy_units)
    
    # Values
    values = df['ENTRIESn_hourly']
    m = len(values)

    features, mu, sigma = normalize_features(features)
    features['ones'] = np.ones(m) # Add a column of 1s (y intercept)
    
    # Convert features and values to numpy arrays
    features_array = np.array(features)
    values_array = np.array(values)

    # Set values for alpha, number of iterations.
    alpha = 0.25 # please feel free to change this value
    num_iterations = 100 # please feel free to change this value

    # Initialize theta, perform gradient descent
    theta_gradient_descent = np.zeros(len(features.columns))
    theta_gradient_descent, cost_history = gradient_descent(features_array, 
                                                            values_array, 
                                                            theta_gradient_descent, 
                                                            alpha, 
                                                            num_iterations)
    
    plot = None
    # -------------------------------------------------
    # Uncomment the next line to see your cost history
    # -------------------------------------------------
    plot = plot_cost_history(alpha, cost_history)
    # 
    # Please note, there is a possibility that plotting
    # this in addition to your calculation will exceed 
    # the 30 second limit on the compute servers.
    
    predictions = np.dot(features_array, theta_gradient_descent)
    return predictions, plot


def plot_cost_history(alpha, cost_history):
   """This function is for viewing the plot of your cost history.
   You can run it by uncommenting this

       plot_cost_history(alpha, cost_history) 

   call in predictions.
   
   If you want to run this locally, you should print the return value
   from this function.
   """
   cost_df = pandas.DataFrame({
      'Cost_History': cost_history,
      'Iteration': range(len(cost_history))
   })
   return ggplot(cost_df, aes('Iteration', 'Cost_History')) + \
      geom_point() + ggtitle('Cost History for alpha = %.3f' % alpha )


predictions(df)

(array([-1117.06687706,  -629.70384943,   345.02220582, ...,    59.31488759,
          546.67791522,  1034.04094285]), <ggplot: (-898306902)>)

<img src="regress.png">

In [130]:
#Computing R squared
features = df[['rain', 'precipi', 'hour', 'meanwspdi']]
    
dummy_units = pandas.get_dummies(df['UNIT'], prefix='unit')
features = features.join(dummy_units)
    
values = df['ENTRIESn_hourly']
m = len(values)

features, mu, sigma = normalize_features(features)
features['ones'] = np.ones(m) # Add a column of 1s (y intercept)
    
    # Convert features and values to numpy arrays
features_array = np.array(features)
values_array = np.array(values)

    # Set values for alpha, number of iterations.
alpha = 0.25 # please feel free to change this value
num_iterations = 100 # please feel free to change this value

    # Initialize theta, perform gradient descent
theta_gradient_descent = np.zeros(len(features.columns))
theta_gradient_descent, cost_history = gradient_descent(features_array, 
                                                            values_array, 
                                                            theta_gradient_descent, 
                                                            alpha, 
                                                            num_iterations)
    
    
predictions = np.dot(features_array, theta_gradient_descent)


data=df['ENTRIESn_hourly']
mn = np.mean(data)
r_squared=1-(np.sum(np.square(data-predictions))/np.sum(np.square(data-mn)))

print "R squared= ",r_squared
print predictions

R squared=  0.461500534568
[-1117.06687706  -629.70384943   345.02220582 ...,    59.31488759
   546.67791522  1034.04094285]


In [91]:
print sm.graphics.plot_partregress('rain', 'hour',['tempi', 'pressurei','EXITSn_hourly'],data=df, obs_labels=False)

Figure(640x480)
